In [1]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as mlp
import matplotlib.pyplot as plt
import geopandas as gp
import os

%matplotlib inline
plt.style.use('ggplot')
mlp.rcParams['figure.figsize'] = [8.0, 8.0]

In [2]:
base_dir = "../../../../Outcome Measurement Data/MALARIA/Distribucion de MTILD/"

In [3]:
entregas = None
entregasDetalles = None
for year in [2015, 2016, 2017, 2018]:
    for (dirpath, dirnames, filenames) in os.walk(base_dir + "BD " + str(year) + "/"):
        for filename in filenames:
            if "EntregaPIIs." in filename:
                if (filename.endswith(".csv")):
                    temp = pd.read_csv(dirpath + "/" + filename)
                elif (filename.endswith(".xlsx")):
                    temp = pd.read_excel(dirpath + "/" + filename, 0)
                temp["bd_year"] = year
                if entregas is None:
                    entregas = temp
                else:
                    entregas = pd.concat([entregas, temp]) 
            elif "EntregaPIIsDetalle." in filename:
                if (filename.endswith(".csv")):
                    temp = pd.read_csv(dirpath + "/" + filename)
                elif (filename.endswith(".xlsx")):
                    temp = pd.read_excel(dirpath + "/" + filename, 0)
                temp["bd_year"] = year
                if entregasDetalles is None:
                    entregasDetalles = temp
                else:
                    entregasDetalles = pd.concat([entregasDetalles, temp]) 

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [4]:
entregas["year"] = pd.to_numeric(entregas.FechaEnt.astype(str)\
            .map(lambda x: (x[0:4] if x.startswith("201") else x[-4:])), errors= "coerce")
entregas["month"] = pd.to_numeric(entregas.FechaEnt.astype(str)\
            .map(lambda x: (x[5:7] if x.startswith("201") else x[-7:-5])), errors= "coerce")

In [5]:
entregas.month.value_counts(dropna=False)

 6.0     20046
 11.0    19876
 10.0    19499
 12.0    15421
 5.0     15194
 9.0     14296
 4.0     14004
 8.0     12860
 7.0     11775
 3.0      4924
 2.0      4331
 1.0      3544
NaN        124
Name: month, dtype: int64

In [6]:
entregas[(entregas.year == 2016) & (entregas.bd_year == 2016)].CodDepto.value_counts()

16.0    2546
17.0    1998
5.0     1424
18.0    1194
11.0    1131
12.0     975
14.0     775
6.0      558
9.0      522
19.0     449
21.0     362
10.0     138
7.0       85
22.0      74
13.0      46
4.0       14
Name: CodDepto, dtype: int64

In [7]:
entregas[entregas.Pabellones.isna() ==False]

,Apellido,Apellido_2,AñoProy,Camas,Categoria,Categoría,Cedula,CodBoleta,CodComun,CodDepto,...,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,bd_year,lp,nomlp,year,month


In [8]:
entregas[(entregas.year == 2016) & (entregas.bd_year == 2017)].CodDepto.value_counts()

16.0    2546
17.0    1998
5.0     1424
18.0    1194
14.0     775
9.0      522
19.0     456
10.0     138
7.0       85
Name: CodDepto, dtype: int64

In [9]:
entregas.columns

Index(['Apellido', 'Apellido_2', 'AñoProy', 'Camas', 'Categoria', 'Categoría',
       'Cedula', 'CodBoleta', 'CodComun', 'CodDepto', 'CodMuni', 'CodReg',
       'FechaEnt', 'Firma', 'Huella', 'MEmbarazada', 'Menores5a', 'Nombre',
       'Nombre_2', 'OID_', 'Pabellones', 'Personas', 'ProyectoMalaria',
       'Responsable', 'Sustit', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1',
       'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'bd_year', 'lp', 'nomlp',
       'year', 'month'],
      dtype='object')

In [10]:
entregas_sub = entregas[((entregas.bd_year==2015) & (entregas.year <= 2015)) | 
                 ((entregas.bd_year==2016) & (entregas.year == 2016)) |
                 ((entregas.bd_year==2017) & (entregas.year == 2017)) |
                 ((entregas.bd_year==2018) & (entregas.year == 2018))]

In [11]:
(entregas.bd_year).value_counts().sort_index()

2015    37453
2016    42095
2017    38080
2018    38266
Name: bd_year, dtype: int64

In [12]:
eMerge = pd.merge(entregas_sub[["CodDepto", "CodComun", "CodBoleta", "bd_year", "FechaEnt", "year", "month"]], 
                  entregasDetalles[["CodDepto", "CodComun", "CodBoleta", "bd_year", 
                                    "Personas", "MEmbarazada", "Menores5a", "Camas", "Pabellones", "Sustit"]], 
                  how="inner", 
                  on=["CodComun", "CodBoleta", "bd_year"], suffixes=("_e", "_eD"))

In [13]:
eMerge.shape

(567243, 14)

In [14]:
entregasDetalles.shape

(1530109, 30)

In [15]:
entregasDetalles.bd_year.value_counts()

2016    420840
2017    394937
2015    364985
2018    349347
Name: bd_year, dtype: int64

In [16]:
eMerge = eMerge[eMerge.year.isna() == False]

In [19]:
eMerge[eMerge.Sustit == False].groupby(["CodDepto_e", (eMerge.year)]).\
    Pabellones.sum().unstack(1).fillna(0)

year,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0
CodDepto_e,,,,,,,
2.0,0.0,0.0,0.0,5137.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,2500.0,346.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,2103.0,5307.0
6.0,0.0,0.0,11907.0,8650.0,18250.0,0.0,0.0
7.0,0.0,0.0,0.0,6070.0,1936.0,422.0,0.0
9.0,0.0,0.0,0.0,23337.0,14199.0,446.0,1147.0
10.0,63792.0,47455.0,0.0,45613.0,57246.0,59.0,0.0
11.0,29612.0,28303.0,0.0,45595.0,28365.0,0.0,24320.0
12.0,19487.0,20463.0,50.0,24978.0,20500.0,0.0,0.0


In [46]:
deptos = gp.read_file("../../../../Covariates and Other Data/GIS/GT-IGN-cartografia_basica-Departamentos.geojson")
deptos = deptos[lambda x: x.CODIGO.astype(int) <= 2200]

In [ ]:
mlp.rcParams['figure.figsize'] = [10.0, 10.0]
mlp.rcParams['axes.titlesize'] = 20
temp = deptos.merge(eMerge.groupby("CodDepto_e").Pabellones.sum().divide(1000).reset_index()\
                         .assign(deptocode=lambda x: (x.CodDepto_e * 100).astype(int).astype(str)\
                                 .map(lambda y: y if y != 1 else "101")), 
                 left_on="CODIGO", right_on = "deptocode", how="outer" ).fillna(0)
ax = temp[temp.Pabellones == 0].plot(color="grey")
temp[temp.Pabellones > 0].plot("Pabellones", ax = ax, cmap = "Blues", legend=True, edgecolor="black")
ax.axis('off')
ax.set_title("Pabellones entregados entre 2012 y 2017 \n(en miles)")
    


In [64]:
pblYears = eMerge.groupby(["CodDepto_e", "year"]).Pabellones.sum().divide(1000).reset_index()\
                         .assign(deptocode=lambda x: (x.CodDepto_e * 100).astype(int).astype(str)\
                                 .map(lambda y: y if y != 1 else "101"))

In [ ]:
vmax = pblYears.Pabellones.max()

side = 8
n = 7
vmin = 0
#vmax = 8
f, axes = plt.subplots(int(np.ceil(n/2)), 2, figsize=(side*2*1.1, side*np.ceil(n/2) ))
i = 2012
for row in axes:
    for ax in row:
        temp3 = deptos.merge(pblYears[pblYears.year == i], 
                             left_on="CODIGO", right_on = "deptocode", how="left" ).fillna(0)
        temp3[temp3.Pabellones == 0].plot(ax=ax, color="grey")
        if len(temp3[temp3.Pabellones > 0])>0:
            temp3[temp3.Pabellones > 0].plot("Pabellones", ax = ax, cmap = "Blues", 
                                             legend=True, edgecolor="black",
                                             vmin = vmin, vmax = vmax)
        ax.axis('off')
        ax.set_title("Malaria bednets delivered\nThousands by department for "+ str(i))
        i += 1

In [66]:
eMerge.to_csv("../../../../Outcome Measurement Data/MALARIA/GTM - Bednets Distr.csv")

In [67]:
eMerge[(eMerge.year==2016) & (eMerge.CodDepto_e == 5)].Pabellones.sum()

41708.0

In [57]:
pblYears[pblYears.year==2018]

,CodDepto_e,year,Pabellones,deptocode
